## Рекомендательные системы 

## Коллаборативная фильтрация» / ПАКЕТ SURPRISE


- используйте данные MovieLens 1M
- можно использовать любые модели из пакета
- RMSE посчитать на основе CrossValidation (5 фолдов)

получите RMSE на тестовом сете 0.87 и ниже

### 0.  Цели работы:
Исследование взаимосвязей следующих показателей базы Movielens:
    - насколько связан состав тэгов пользователя с уровнем оценкок фильмов именно этого пользователя (спойлер: не связан)
    - насколько связан состав тэгов (+ жанр + год выпуска фильма) в отошении фильма (в целом)  со средней (медианой) оценкой этого фильма  (спойлер: связан и хорошо)

Насколько состав и качество данных а также уровень статистической связанности пригоден для создания рекомендательной системы
2. Тестирование пакета Surprise ия поставленных задач
3. Выход в тесте на минимально возможное значение RMSE по любой из рассматриваемых моделей

### 1. Импорт библиотек / данных 

In [ ]:
import pandas as pd
import numpy as np
import re

from warnings import filterwarnings 
filterwarnings('ignore')

In [ ]:
from surprise import SVD, SVDpp, SlopeOne
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import cross_validate,  GridSearchCV

### 2. Предобработка данных и конструирование признаков

#### 2.1  Movies

In [1]:
movies = pd.read_csv('movies.csv')
movies.head(1)

NameError: ignored

Представим себе, что каждый жанр, указанный в поле genres -  это тэг,  и сделаем:

In [ ]:
genres_vect = movies["genres"].str.get_dummies("|").iloc[:,1:]

In [ ]:
genres_vect.head(1)

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
i             = 0
col_s         = ['movieId','tag']
genres_as_tag = pd.DataFrame(columns =col_s)


for i in range (19):
    df                      = pd.DataFrame(list(zip(movies.movieId, genres_vect.iloc[:,i])))
    df.columns              = col_s
    df.loc[df.tag>0, 'tag'] = genres_vect.columns[i]
    genres_as_tag           = pd.concat([genres_as_tag, df],axis = 0)

genres_as_tag = genres_as_tag.loc[genres_as_tag.tag != 0]

In [ ]:
genres_as_tag.head(1)

,movieId,tag
5,6,Action


сформируем дополнительные признаки - год выпуска фильма и период выпуска фильма (с точностью до 5-10 лет)

In [ ]:
def set_year(row):
    z = re.findall('(\d\d\d\d)', row.title)
    if len(z) == 0 or z == 0 or z == None:
        z = 9999
    else:
        z = int(z[-1])
    return z    

movies['year_of_film']= movies.apply(set_year, axis=1)

In [ ]:
movies['epoch_of_film'] = pd.cut(movies.year_of_film, [1900, 1960, 1970, 1980, 1990, 2000,
                                                       2005, 2010, 2015, 10000]) 
movies['epoch_of_film'].value_counts()

(2010, 2015]     10713
(2005, 2010]      8176
(1900, 1960]      8040
(1990, 2000]      7062
(2000, 2005]      5417
(2015, 10000]     5295
(1980, 1990]      4933
(1970, 1980]      4738
(1960, 1970]      3605
Name: epoch_of_film, dtype: int64

Эпоха фильма тоже может быть тэгом, почему бы и нет:

In [ ]:
epoch_as_tag = movies.iloc[:,[0,4]]
epoch_as_tag.columns  = col_s
epoch_as_tag.head()

,movieId,tag
0,1,"(1990, 2000]"
1,2,"(1990, 2000]"
2,3,"(1990, 2000]"
3,4,"(1990, 2000]"
4,5,"(1990, 2000]"


Соединим все синтетические тэги и оставим до лучших временЖ

In [ ]:
movie_tag = pd.concat([genres_as_tag, epoch_as_tag],axis = 0)
movie_tag.head()

,movieId,tag
5,6,Action
8,9,Action
9,10,Action
14,15,Action
19,20,Action


In [ ]:
movie_tag.shape

(159939, 2)

#### 2.2  tags

In [ ]:
tags = pd.read_csv('tags.csv')

In [ ]:
tags.head()

,userId,movieId,tag,timestamp
0,14,110,epic,1443148538
1,14,110,Medieval,1443148532
2,14,260,sci-fi,1442169410
3,14,260,space action,1442169421
4,14,318,imdb top 250,1442615195


In [ ]:
tags.tag.value_counts().head()

sci-fi         9400
atmospheric    6430
action         6219
comedy         5923
surreal        5299
Name: tag, dtype: int64

исключаем пустые записи

In [ ]:
tags = tags[~tags.tag.isna()]

приводим все теги к нижнему регистру

In [ ]:
tags['tag'] = tags['tag'].str.lower()

если наименование жанра содержится в теге - оставляем типовое поле о принадлежности тега к жанру, жанр уже отражен в базе movies тэгов

In [ ]:
for i in genres_vect.columns.str.lower().tolist():
    tags.loc[tags.tag ==i, 'tag'] = 'genres_tag'

In [ ]:
tags.tag.value_counts().head()

genres_tag         60851
atmospheric         6995
surreal             5572
funny               5560
based on a book     5414
Name: tag, dtype: int64

если время выпуска фильма содержится в теге - оставляем типовое поле о принадлежности тега ко времение, "эпоха" уже отражена в базе movies

In [ ]:
tags[tags.tag.str.contains('0s')].head()

,userId,movieId,tag,timestamp
292,206,541,80s,1527550262
293,206,2115,80s,1527550065
295,206,2402,80s,1527652380
296,206,2403,80s,1527652469
297,206,3740,80s,1527550214


In [ ]:
tags.loc[tags.tag.str.contains('0s'),  'tag'] = 'epoch_tag'
tags.loc[tags.tag.str.contains("0's"), 'tag'] = 'epoch_tag'

В списке тэгов очень часто встречаются имена и фамилии. Это либо имена актеров и создателей фильма или имена главных героев фильма. Введем универсальный признак star_tag, обозначающий, что в тэге есть имя Звезды

In [ ]:
men_names    = pd.read_html('https://audio-class.ru/names/mens-names.php', 
                         encoding = 'utf-8')[0].iloc[:,1].str.lower().tolist()
women_names  = pd.read_html('https://audio-class.ru/names/womens-names.php/', 
                        encoding = 'utf-8')[0].iloc[:,1].str.lower().tolist()
family_names = pd.read_html('https://audio-class.ru/names/last-names-sortable.php', 
                        encoding = 'utf-8')[0].iloc[:,1].str.lower().tolist()

names = men_names + women_names + family_names

In [ ]:
n = []
for i in names:
    n += (i.split('/'))
names = n

In [ ]:
tags.loc[tags.tag.str.contains('|'.join(names)), 'tag'] = 'star_tag'

In [ ]:
tags[tags.tag=='star_tag'].shape

(365807, 4)

In [ ]:
tags.shape, tags.tag.unique().shape

((1108981, 4), (38174,))

Разберемся со словосочетаниями в тэгах, которые встречаются не более 20 раз, 

логика преобразований:
- составляем "мешок" слов из всех словосочетаний "редких тегов" (встречающихся не чаще 20 раз)
- выбираем Топ 100 самых часто встречающихся слов "из мешка",
- убираем из Топ 100 очевидные предлоги и слова без самостоятельной смысловой нагрузки,
- если в "редком теге" встречается слово из Топ100, заменяем "редкий тэг" на ключевое слово редкого тэга

In [ ]:
rare_tags = tags.tag.value_counts()[tags.tag.value_counts() < 20]
rare_tags

doping                   19
otto preminger           19
institutions             19
auschwitz                19
movies 8                 19
                         ..
sms                       1
red queen                 1
star war                  1
works of pure fiction     1
dana delany               1
Name: tag, Length: 34552, dtype: int64

In [ ]:
rti = tuple(rare_tags.index)

In [ ]:
l =[]
for i in rti:
    l.append(re.findall('[\w]+', i))

In [ ]:
from collections import Counter

In [ ]:
l1 = []
for i in Counter(sum(l, [])).most_common(100):
    l1.append(i[0])
l1[:10]

['the', 'of', 'a', 'movie', 'to', 'in', 's', 'and', 'writer', 'not']

In [ ]:
words_bag=('movie', 'writer', 'bad', 'not', 'film', 'great', 'man', 'no', 'director', 'story', 'music', 'ending',
          'plot', 'child', 'book', 'family', 'character', 'woman', 'setting', 'too', 'death', 'sex', 'see', 'funny',
          'afi', 'time', 'based', 'characters', 'better', 'girl', 'up', 'scene', 'relationship', 'animal', 'american',
          'classic', 'one', 'all', 'world', 'self', 'new', 'car', 'protagonist', 'robert', 'you', 'evil', 'than',
          'watch', 'women', 'but', 'soundtrack', 'old', 'de', 'stupid', 'humor', 'human', 'out', 'poor',  'very',
          '13', 'father', 'acting', 'real', 'sexual', 'fight', 'original', 'history', 'boy', 'mother', 'dead', '06', 
          'art', 'hero', 'first', 'kids', 'drug')

In [ ]:
for i in range(tags.shape[0]): 
    if tags.iloc[i,2] in rti:
        for w in words_bag:
            if w in tags.iloc[i,2]:
                tags.iloc[i,2] = w

In [ ]:
tags.tag.value_counts().head()

star_tag       365807
genres_tag      60851
atmospheric      6995
de               5903
funny            5860
Name: tag, dtype: int64

Далее ограничиваем выборку по следующим направлениям, рассматриваем :
- только ТОП 1000 самых популярных тэгов,
- только ТОП 1000 самых активных юзеров,

иначе упираемя в ограничения по оперативной памяти

In [ ]:
limit = 1000

In [ ]:
top_tags      = tags.tag.value_counts().index[:limit]
selected_tags = tags.loc[tags.tag.str.contains('|'.join(top_tags)) ]
selected_tags.shape

(1052650, 4)

In [ ]:
sum_tags = selected_tags.iloc[:,:3]

In [ ]:
sum_tags.head(1)

,userId,movieId,tag
0,14,110,epic


#### 2.3.  ratings

In [ ]:
ratings = pd.read_csv('ratings.csv')
ratings.head(1)

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221


In [ ]:
top_users        = ratings.userId.value_counts().index[:limit]
selected_ratings = ratings.loc[
                               ratings.userId.astype(str).str.contains
                               ('|'.join(top_users.astype(str)))]
selected_ratings.shape

(3057290, 4)

и выходим на аккуратную базу, в которой есть только сведения для предсказания медианного рейтинга

In [ ]:
base_surprise = selected_tags.merge(selected_ratings.iloc[:,:3], 
                                    how = 'left', 
                                    left_on = ['userId', 'movieId'], 
                                    right_on= ['userId', 'movieId'])
base_surprise.head(1)

,userId,movieId,tag,timestamp,rating
0,14,110,epic,1443148538,NaN


In [ ]:
base_surprise = base_surprise[~base_surprise.rating.isna()]

In [ ]:
base_surprise.shape

(190563, 5)

In [ ]:
base_surprise.rating.min(), base_surprise.rating.max()

(0.5, 5.0)

### 3. Построение моделей Surprise

попробуем предсказать для конкретного Пользователя и поставленным тегам - средний рейтинг этого фильма

In [ ]:
dataset = pd.DataFrame({
    'uid':    base_surprise.userId,
    'iid':    base_surprise.tag,
    'rating': base_surprise.rating
})

In [ ]:
reader      = Reader(line_format = 'user item rating', 
                     rating_scale= (0.5,5.0))

df_prepared = Dataset.load_from_df(dataset, reader)

In [ ]:
algo_svd = SVD(n_factors=20, n_epochs=20)

In [ ]:
cv_svd = cross_validate(algo_svd, 
                        df_prepared, 
                        measures=['RMSE'], 
                        cv=5, 
                        verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0174  1.0177  1.0202  1.0140  1.0262  1.0191  0.0040  
Fit time          3.90    3.88    3.80    3.86    3.77    3.84    0.05    
Test time         0.56    0.20    0.52    0.20    0.51    0.40    0.16    


SlopeOne

In [ ]:
algo_SlopeOne = SlopeOne()
cv_SlopeOne   = cross_validate(algo_SlopeOne, 
                               df_prepared, 
                               measures=['RMSE'], 
                               cv=5, verbose=True)

Evaluating RMSE of algorithm SlopeOne on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0106  1.0010  1.0073  1.0094  1.0150  1.0087  0.0046  
Fit time          46.18   39.40   47.37   40.24   44.84   43.61   3.20    
Test time         127.05  124.10  125.64  124.40  127.67  125.77  1.41    


Surprise - sequel

+ добавляем тэги по movies
+ тэги включаем по фильтру : если частота упоминария больше 20
+ unit = movieid
+ рейтинг считается, как медиана по фильму

In [ ]:
selected_tags_2=tags[~tags.tag.isin(
                    tags.tag.value_counts()[tags.tag.value_counts() <= 20]
                    .index)].iloc[:,[1,2]]
selected_tags_2.head(1)

,movieId,tag
0,110,epic


In [ ]:
sum_tags = pd.concat([movie_tag, selected_tags_2], axis = 0)
sum_tags.head(1)

,movieId,tag
5,6,Action


In [ ]:
ratings_avg = pd.read_csv('ratings.csv'
                        ).groupby(['movieId']
                        ).rating.median()

In [ ]:
base_surprise_2 = sum_tags.merge(ratings_avg, 
                                 how = 'left', 
                                 left_on = ['movieId'], 
                                 right_on= ratings_avg.index)
base_surprise_2.head(1)

,movieId,tag,rating
0,6,Action,4.0


In [ ]:
base_surprise_2 = base_surprise_2[~base_surprise_2.rating.isna()]

In [ ]:
dataset_2 = pd.DataFrame({
    'uid':    base_surprise_2.movieId,
    'iid':    base_surprise_2.tag,
    'rating': base_surprise_2.rating
})

In [ ]:
reader_2      = Reader(line_format='user item rating', 
                       rating_scale=(0.5,5.0))

df_prepared_2 = Dataset.load_from_df(dataset_2, reader_2)

In [ ]:
algo_svd_2 = SVD(n_factors=20, n_epochs=20)

cv_svd_2   = cross_validate(algo_svd_2, 
                            df_prepared_2, 
                            measures=['RMSE'], 
                            cv=5, 
                            verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.2988  0.2847  0.2929  0.2765  0.2813  0.2868  0.0080  
Fit time          23.52   23.94   24.21   23.98   24.03   23.94   0.23    
Test time         2.23    2.28    2.24    2.62    2.24    2.32    0.15    


In [ ]:
algo_SlopeOne_2 = SlopeOne()
cv_SlopeOne_2   = cross_validate(algo_SlopeOne_2, 
                                 df_prepared_2, 
                                 measures=['RMSE'], 
                                 cv=5, 
                                 verbose=True)

Evaluating RMSE of algorithm SlopeOne on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.0751  0.0736  0.0737  0.0775  0.0738  0.0748  0.0015  
Fit time          19.48   19.42   19.51   19.42   19.38   19.44   0.05    
Test time         85.02   79.87   80.29   80.94   78.67   80.96   2.16    


Задача минимум

In [ ]:
dataset_3 = pd.DataFrame({
    'uid':    base_surprise.userId,
    'iid':    base_surprise.movieId,
    'rating': base_surprise.rating
})

In [ ]:
reader_3      = Reader(line_format='user item rating', 
                       rating_scale=(0.5,5.0))

df_prepared_3 = Dataset.load_from_df(dataset_3, reader_3)

In [ ]:
algo_svd_3 = SVD(n_factors=20, n_epochs=20)

cv_svd_3   = cross_validate(algo_svd_3, 
                            df_prepared_3, 
                            measures=['RMSE'], 
                            cv=5, 
                            verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.4306  0.4249  0.4322  0.4266  0.4219  0.4272  0.0037  
Fit time          5.98    6.20    5.95    5.80    5.44    5.87    0.25    
Test time         0.75    0.37    2.63    0.28    0.71    0.95    0.86    
